In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

from numpy import cos, sin
import subprocess
import os

In [2]:
outDir = "frames"
FFMPEG = 'ffmpeg'
numFrames = 60
CMD = [FFMPEG, '-y', '-r', '30', '-f', 'image2', '-i', f'{outDir}/frame_%03d.png',
       '-vcodec', 'libx264', '-crf', '25', '-pix_fmt', 'yuv420p', 'isometric.mp4']
print(' '.join(CMD))

def animate():
    subprocess.run(CMD)



ffmpeg -y -r 30 -f image2 -i frames/frame_%03d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p isometric.mp4


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
from math import cos, sin, radians

outDir = "frames"
os.makedirs(outDir, exist_ok=True)

def draw_checkerboard(ax, iso, grid_density=1, grid_range=(-20, 20)):
    x = np.arange(grid_range[0], grid_range[1] + grid_density, grid_density)
    y = np.arange(grid_range[0], grid_range[1] + grid_density, grid_density)
    
    for i in range(len(x) - 1):
        for j in range(len(y) - 1):
            color = 'white' if (i + j) % 2 == 0 else 'black'
            square = np.array([[x[i], 0, y[j]], [x[i], 0, y[j+1]], [x[i+1], 0, y[j+1]], [x[i+1], 0, y[j]]])
            square_iso = isometric(square, iso)
            ax.fill(square_iso[:, 0], square_iso[:, 1], color=color)

def isometricMatrix(alpha, beta):
    mX = np.array([
        [1, 0, 0],
        [0, cos(alpha), sin(alpha)],
        [0, -sin(alpha), cos(alpha)],
    ])
    mY = np.array([
        [cos(beta), 0, -sin(beta)],
        [0, 1, 0],
        [sin(beta), 0, cos(beta)]
    ])
    return np.dot(mX, mY)

def isometric(vertices, iso):
    return np.dot(vertices, iso.T)

def create_isometric_grid(grid_density=1, grid_range=(-5, 5)):
    grid_lines = []
    for x in np.arange(grid_range[0], grid_range[1] + 1, grid_density):
        for z in np.arange(grid_range[0], grid_range[1] + 1, grid_density):
            p1 = np.array([x, 0, z])
            p2 = np.array([x + grid_density, 0, z + grid_density])
            grid_lines.append((p1, p2))
    return grid_lines

def draw_isometric_grid(ax, grid_lines, iso):
    for p1, p2 in grid_lines:
        p1_iso = isometric(np.array([p1]), iso)
        p2_iso = isometric(np.array([p2]), iso)
        ax.plot([p1_iso[0, 0], p2_iso[0, 0]], [p1_iso[0, 1], p2_iso[0, 1]], 'gray', linestyle='--', lw=0.5)

def rotationMatrixX(theta):
    # return np.array([
    #     [1, 0, 0],
    #     [0, cos(theta), -sin(theta)],
    #     [0, sin(theta), cos(theta)]
    # ])
    # return np.array([
    #     [cos(theta), -sin(theta), 0],
    #     [sin(theta), cos(theta), 0],
    #     [0, 0, 1 ]
    # ])
    return np.array([
        [cos(theta), 0, -sin(theta)],
        [0, 1, 0],
        [sin(theta), 0, cos(theta) ]
    ])

side_length = 2.0
height = 1.0
depth = 1.0
vertices = np.array([
    [-side_length / 2, -height / 2, -depth / 2],
    [side_length / 2, -height / 2, -depth / 2],
    [side_length / 2, height / 2, -depth / 2],
    [-side_length / 2, height / 2, -depth / 2],
    [-side_length / 2, -height / 2, depth / 2],
    [side_length / 2, -height / 2, depth / 2],
    [side_length / 2, height / 2, depth / 2],
    [-side_length / 2, height / 2, depth / 2],
])

faces = np.array([
    [0, 1, 2, 3],  
    [4, 5, 6, 7],  
    [0, 1, 5, 4],  
    [1, 2, 6, 5], 
    [2, 3, 7, 6],  
    [3, 0, 4, 7],
])

alpha, beta = radians(30), radians(120)
iso = isometricMatrix(alpha, beta)

x_limits = (-5, 5)
y_limits = (-5, 5)
grid_limits = (-10, 10)
grid_lines = create_isometric_grid(grid_density=1, grid_range=grid_limits)

numFrames = 60
for i in range(numFrames):
    plt.figure(figsize=(8, 8))
    ax = plt.gca()
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)
    
    draw_checkerboard(ax, iso)
    
    tx = i * 0.1 
    ty = i * 0.1
    tz = 1
    translatedVertices = vertices + np.array([tx, ty, tz])
    
    theta = np.radians(i*10)
    R = rotationMatrixX(theta)
    
    for face in faces:
        faceVert = translatedVertices[face]
        rotatedVertices = np.dot(R, faceVert.T).T
        isoVertices = isometric(rotatedVertices, iso)
        plt.plot(isoVertices[:, 0], isoVertices[:, 1], 'b-')
    
    plt.xlabel('$O_x$')
    plt.ylabel('$O_y$')
    plt.title(f"Frame {i}")
    plt.xlim(-10,10)
    plt.ylim(-10,10)
    
    plt.grid(False)
    plt.savefig(f"{outDir}/frame_{i:03d}.png")
    plt.close()

def animate():
    FFMPEG = 'ffmpeg'
    CMD = [FFMPEG, '-y', '-r', '30', '-f', 'image2', '-i', f'{outDir}/frame_%03d.png',
           '-vcodec', 'libx264', '-crf', '25', '-pix_fmt', 'yuv420p', 'parallelepiped_in_projection.mp4']
    subprocess.run(CMD)

animate()


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab